In [1]:
from openai import OpenAI
import os
import pandas as pd
from sklearn.metrics import f1_score


In [ ]:
api_key = "<<<YOUR GPT API KEY>>>"

In [3]:

# API 키를 환경 변수에 저장
os.environ['OPENAI_API_KEY'] = api_key

def create_chat_completion(system_input, user_input, model="gpt-4o-mini", temperature=1.15, max_tokens=150):
    try:
        # 메시지 목록을 자동으로 생성해요
        messages = [
            {"role": "system", "content": system_input},
            {"role": "user", "content": user_input}
        ]

        response = OpenAI().chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens  # 최대 토큰 수를 지정해요
        )
        # 생성된 응답을 반환해요
        return response
    except Exception as e:
        return f"Error: {str(e)}"

# System

In [4]:
#train을 사용하려 여러 시도를 해봤으나 성능을 향상시키지 못함
system_input = """You are an expert in automotive data. You will be given data like below and you have to predict if it is automotive-related data or not. If it is automotive-related data, determine it as 1, otherwise 0.
example data looks like this and you have to predict answer for user input
ID: TRAIN_01
title: 도로교통공단_고속도로구간별 도로위험도지수정보 조회 서비스
notes: 고속도로명, 고속도로구간명과 차종코드를 이용하여 구간 내의 도로위험도지수값, 도로위험도등급 등의 구간좌표별 실시간 도로위험도지수정보를 제공하는 서비스
answer: 0

ID: TRAIN_02
title: New registrations of road vehicles by vehicle group and type
notes: This dataset presents the monthly data of new registrations of road vehicles (passenger cars, passenger vehicles, goods vehicles, agricultural vehicles, industrial vehicles, motorcycles, trailers) by canton and fuel, since 2005.
answer: 1

so your output should be like this: 0\n1
You are an expert in automotive data. Follow these instructions:

1. If the input data is automotive-related data, determine it as 1, otherwise 0. There are 40 data to answer.
2. ID: TEST_00 ~ TEST_39, and answer as 0 or 1 for each data, separated by row.
3. Be aware of output you can only contrain 0 or 1 for each data and '\n'for seperate this. If you output is wrong you have to go back and make output again
"""

In [5]:
#ver 1 
system_input = """You are an expert in automotive data. Follow these instructions:

1. If the input data is automotive-related data, determine it as 1, otherwise 0. There are 40 data to answer.
2. ID: TEST_00 ~ TEST_39, and answer as 0 or 1 for each data, separated by row.
3. Be aware of output you can only contain '0' or '1' for each data and '\n'for seperate this. If you output is wrong you have to go back and make output again
"""

In [6]:
system_input = """You are an expert in automotive data. Follow these instructions:

1. If the input data is automotive-related data, determine it as 1, otherwise 0. There are 40 data to answer.
2. ID: TEST_00 ~ TEST_39, and answer as 0 or 1 for each data, separated by row.
3. Each Test data has a title. You have to use these information for prediction.
4. If unsure or if your score is low, backtrack and try a different approach.
5. Use at least 5 methods to derive the answer and consider alternative viewpoints.
"""

In [7]:
system_input = """You are an expert in automotive data. Follow these instructions:

0. First translate the title to English.
1. If the input data is automotive-related data, determine it as 1, otherwise 0. There are 40 data to answer.
2. ID: TEST_00 ~ TEST_39, and answer as 0 or 1 for each data, separated by row.
3. Each Test data has a lang and title. You have to use these information for prediction.
4. If unsure or if your score is low, backtrack and try a different approach.
5. Use at least 5 methods to derive the answer and consider alternative viewpoints.
"""

# User

In [8]:
test = pd.read_csv("/home/yhkim/data/test.csv")

In [9]:
test.head(10)

,ID,lang,title,notes,answer
0,TEST_00,en,Beach Profile Data Collected from Madeira Beac...,"This dataset, prepared by the U.S. Geological ...",0
1,TEST_01,en,Nota media de la nota de admisión (cohorte de ...,Tabla de EDUCAbase Nota media de la nota de ad...,0
2,TEST_02,en,Internet-based platform services 2019 accordin...,NaN,0
3,TEST_03,kr,경상남도 김해시_자동차등록 현황,"경상남도 김해시 자동차등록 현황에 대한 데이터로 연도별 총계, 승용차, 승합차, 화...",1
4,TEST_04,jp,EV用充電設備の設置状況データ一覧,【東京都環境局】2021年3月末時点の都内におけるEV用充電設備の情報です。【リソース】EV...,0
5,TEST_05,kr,경기아트센터 장애인기업 생산제품 구매 현황,경기아트센터에서 제공하는 장애인기업 생산 제품 구매 현황,0
6,TEST_06,de,Autohaus,Weitere Informationen: https://wiki.openstreet...,1
7,TEST_07,kr,2021년 농식품부 국가안전대진단 실행계획,국가안전대진단은 세월호 참사 이후 2015년부터 각종 재난발생이 우려되는 시설에 대...,0
8,TEST_08,kr,경기도 성남시_교통단속 카메라 파일데이터,경기도 성남시 교통단속 카메라 openAPI 서비스에서 제공하는 데이터를 다운 받을...,0
9,TEST_09,kr,영화 관람 데이터,ㅇ 데이터 소개\n- 영화진흥위원회가 운영하는 KOBIS 영화관입장권통합전산망 데이...,0


In [10]:
##train을 사용하는 prompt, 위에서 언급한대로 성능을 향상시키지 못함

user_input = """
###Example###
ID: TRAIN_01
title: 도로교통공단_고속도로구간별 도로위험도지수정보 조회 서비스
answer: 0

ID: TRAIN_02
title: New registrations of road vehicles by vehicle group and type
answer: 1

ID: TRAIN_03
title: Siedlungs- und Verkehrsflächen nach Nutzungsartengruppen in Böxlund
answer: 1

ID: TRAIN_04
title: Marine Geophysical and Seismic Data from around the UK (1966 Onwards)
answer: 0

ID: TRAIN_05
title: Truck Size and Weight Enforcement Data
answer: 1

###User Input###
"""

for index, row in test.iterrows():
    user_input += f"ID: {row['ID']}\n"
    # user_input += f"language: {row['lang']}\n"
    user_input += f"title: {row['title']}\n\n"
    # user_input += f"notes: {row['notes'] if not pd.isna(row['notes']) else ''}\n\n"

In [11]:
test

,ID,lang,title,notes,answer
0,TEST_00,en,Beach Profile Data Collected from Madeira Beac...,"This dataset, prepared by the U.S. Geological ...",0
1,TEST_01,en,Nota media de la nota de admisión (cohorte de ...,Tabla de EDUCAbase Nota media de la nota de ad...,0
2,TEST_02,en,Internet-based platform services 2019 accordin...,NaN,0
3,TEST_03,kr,경상남도 김해시_자동차등록 현황,"경상남도 김해시 자동차등록 현황에 대한 데이터로 연도별 총계, 승용차, 승합차, 화...",1
4,TEST_04,jp,EV用充電設備の設置状況データ一覧,【東京都環境局】2021年3月末時点の都内におけるEV用充電設備の情報です。【リソース】EV...,0
5,TEST_05,kr,경기아트센터 장애인기업 생산제품 구매 현황,경기아트센터에서 제공하는 장애인기업 생산 제품 구매 현황,0
6,TEST_06,de,Autohaus,Weitere Informationen: https://wiki.openstreet...,1
7,TEST_07,kr,2021년 농식품부 국가안전대진단 실행계획,국가안전대진단은 세월호 참사 이후 2015년부터 각종 재난발생이 우려되는 시설에 대...,0
8,TEST_08,kr,경기도 성남시_교통단속 카메라 파일데이터,경기도 성남시 교통단속 카메라 openAPI 서비스에서 제공하는 데이터를 다운 받을...,0
9,TEST_09,kr,영화 관람 데이터,ㅇ 데이터 소개\n- 영화진흥위원회가 운영하는 KOBIS 영화관입장권통합전산망 데이...,0


In [12]:
user_input = ""

for index, row in test.iterrows():
    user_input += f"ID: {row['ID']}\n"
    # user_input += f"lang: {row['lang']}\n"
    user_input += f"title: {row['title']}\n\n"
    # user_input += f"notes: {' '.join(row['notes'].split()[:5]) if not pd.isna(row['notes']) else ''}\n\n"


In [13]:
len(user_input)

2185

In [14]:
response = create_chat_completion(model="gpt-3.5-turbo-0125", temperature=0.4, max_tokens=1000, system_input=system_input, user_input=user_input)

In [15]:
user_input += "make prediction for above 40 datas. answer as 0 or 1 for each data, separated by row(\\n). You dont need to write any other information. If you are not sure about the answer, go back and think again."

In [16]:
print(user_input)

ID: TEST_00
title: Beach Profile Data Collected from Madeira Beach, Florida (January 15, 2021)

ID: TEST_01
title: Nota media de la nota de admisión (cohorte de nuevo ingreso en el SUE) por forma de admisión, ámbito de estudio y sexo (universidades públicas presenciales)

ID: TEST_02
title: Internet-based platform services 2019 according to various criteria

ID: TEST_03
title: 경상남도 김해시_자동차등록 현황

ID: TEST_04
title: EV用充電設備の設置状況データ一覧

ID: TEST_05
title: 경기아트센터 장애인기업 생산제품 구매 현황

ID: TEST_06
title: Autohaus

ID: TEST_07
title: 2021년 농식품부 국가안전대진단 실행계획

ID: TEST_08
title: 경기도 성남시_교통단속 카메라 파일데이터

ID: TEST_09
title: 영화 관람 데이터

ID: TEST_10
title: 인천광역시_차로폭 차로수 현황

ID: TEST_11
title: (1월) 차량용 GPS Raw Data(PND)

ID: TEST_12
title: Liste des jardins partagés à Paris

ID: TEST_13
title: 진주시 CCTV 신호대기시간 데이터

ID: TEST_14
title: Población por unidad poblacional

ID: TEST_15
title: 서울시 자치구별 수소차 등록현황

ID: TEST_16
title: Tablas estadísticas del Censo de Mercado de Trabajo 2021 (CMT). Comarcas y subcomarc

# prediction

In [17]:
for i in range(5):
    response = create_chat_completion(model="gpt-3.5-turbo-0125", temperature=0.4, max_tokens=1000, system_input=system_input, user_input=user_input)
    print(pd.DataFrame(response.choices[0].message.content.split("\n")).transpose())     

    predictions = list(map(int, response.choices[0].message.content.split("\n")))
    #마지막 컬럼이 39인걸 확인할 것

  0  1  2  3  4  5  6  7  8  9   ... 30 31 32 33 34 35 36 37 38 39
0  1  0  0  1  1  0  0  0  1  0  ...  0  1  0  0  0  1  1  0  0  1

[1 rows x 40 columns]
  0  1  2  3  4  5  6  7  8  9   ... 30 31 32 33 34 35 36 37 38 39
0  1  0  0  1  1  0  0  0  1  0  ...  0  1  0  1  1  1  0  1  1  1

[1 rows x 40 columns]
  0  1  2  3  4  5  6  7  8  9   ... 29 30 31 32 33 34 35 36 37 38
0  1  0  0  1  1  0  0  0  1  0  ...  0  0  1  0  0  1  1  1  0  1

[1 rows x 39 columns]
  0  1  2  3  4  5  6  7  8  9   ... 30 31 32 33 34 35 36 37 38 39
0  1  0  0  1  1  0  0  1  1  0  ...  0  1  0  1  1  1  0  1  1  1

[1 rows x 40 columns]
  0  1  2  3  4  5  6  7  8  9   ... 30 31 32 33 34 35 36 37 38 39
0  1  0  0  1  1  0  0  0  1  0  ...  0  0  1  0  1  1  0  1  1  1

[1 rows x 40 columns]


# Make submission

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [19]:
path = "/home/yhkim/code/carPrompt/submission"
submission_base = pd.read_csv(os.path.join(path, "base.csv"))

In [20]:
submission_base

,system,user
0,당신은 자동차 데이터 전문가입니다. 입력으로 들어오는 데이터가 자동차 관련 데이터라...,ID: TEST_00\ntitle: Beach Profile Data Collect...


In [21]:
submission_base["system"] = system_input
submission_base["user"] = user_input


In [22]:
submission_base

,system,user
0,You are an expert in automotive data. Follow t...,ID: TEST_00\ntitle: Beach Profile Data Collect...


In [23]:
submission_base.to_csv(os.path.join(path, "1016.csv"), index=False, encoding='utf-8-sig')